# Pre-process 
Since the datasets SurgVisDom, Giannarou & Mountney represent 88.7% of the data, see [overview](https://docs.google.com/spreadsheets/d/1cIjW4NbDDJmY_u0w-p84z5vhGQlmpcp2JQnmYLdKyL0/edit?usp=sharing), we only process them.

In [2]:
import cv2
import os
import pandas as pd
from tqdm import tqdm

from utils.io import load_yaml, generate_path
from utils.transforms import Compose, dict_list_to_compose
from utils.sampling import ConsecutiveSequences
from utils.processing import RandomEdgeHomography

servers = load_yaml('configs/servers.yml')
server = 'local'

## Create Dataframe
Create sampler over data, apply transforms, safe images as PNG and store consecutive images in Dataframe

In [5]:
# dict of videos
yml = load_yaml(path='configs/high_fps_without_camera_motion_videos_transforms.yml')

absolute_prefix = '/tmp/data'
generate_path(absolute_prefix)

df = pd.DataFrame(columns=['file_sequence', 'path', 'vid_idx', 'frame_idcs', 'database'])

for database in yml['databases']:
    paths = []
    for files in database['videos']['files']:
        paths.append(os.path.join(
            servers[server]['database']['location'], 
            database['prefix'], 
            database['videos']['prefix'], 
            files)
        )

    stride = 1
    composes = []
    for dict_list in database['transforms']:
        composes.append(dict_list_to_compose(dict_list)) # generate compose transform from dict
    consecutive_sequences = ConsecutiveSequences(
        paths=paths, 
        stride=stride, 
        max_seq=10, 
        seq_stride=1, 
        seq_len=3, 
        transforms=composes, 
        verbose=True
    ) # generate iterator
    
    for cs, vid_idx, frame_idx in consecutive_sequences:
        frame_idcs = []
        file_sequence = []
        if database['test'] == True:
            relative_prefix = 'test'
        else:
            relative_prefix = 'train'
        relative_prefix = os.path.join(relative_prefix, database['videos']['files'][vid_idx][:-4]) # unique name of the video

        for frame in cs:
            frame_idcs.append(frame_idx)
            file_name = 'frame_{}.png'.format(frame_idx)
            file_sequence.append(file_name)
            frame_idx += stride

            prefix = os.path.join(absolute_prefix, relative_prefix)
            generate_path(prefix)

            path = os.path.join(prefix, file_name)

            if not os.path.isfile(path):
                cv2.imwrite(path, frame)

        df = df.append({
            'file_sequence': file_sequence, 
            'path': relative_prefix, 
            'vid_idx': vid_idx, 
            'frame_idcs': frame_idcs,
            'database': {'name': database['name'], 'prefix': database['prefix'], 'test': database['test']}
        }, ignore_index=True)

        # cv2.imshow('img', cs[0])
        # cv2.waitKey()
            
df_name = 'log'
df.to_pickle(os.path.join(absolute_prefix, '{}.pkl'.format(df_name)))
df.to_csv(os.path.join(absolute_prefix, '{}.csv'.format(df_name)))

SurgVisDom_test_no_indicators_without_camera_motion train/vid_4
SurgVisDom_test_no_indicators_without_camera_motion train/vid_4
SurgVisDom_test_no_indicators_without_camera_motion train/vid_4
SurgVisDom_test_no_indicators_without_camera_motion train/vid_4
SurgVisDom_test_no_indicators_without_camera_motion train/vid_4
SurgVisDom_test_no_indicators_without_camera_motion train/vid_4
SurgVisDom_test_no_indicators_without_camera_motion train/vid_4
SurgVisDom_test_no_indicators_without_camera_motion train/vid_4
SurgVisDom_test_no_indicators_without_camera_motion train/vid_4
SurgVisDom_test_no_indicators_without_camera_motion train/vid_4
SurgVisDom_test_no_indicators_without_camera_motion test/vid_0
SurgVisDom_test_no_indicators_without_camera_motion test/vid_0
SurgVisDom_test_no_indicators_without_camera_motion test/vid_0
SurgVisDom_test_no_indicators_without_camera_motion test/vid_0
SurgVisDom_test_no_indicators_without_camera_motion test/vid_0
SurgVisDom_test_no_indicators_without_camera_

## Visiualization Examples
Create sampler over data, apply transforms, and visualize

In [9]:
# dict of videos
# yml = load_yaml(path='configs/high_fps_with_camera_motion_videos_transforms.yml')
yml = load_yaml(path='configs/high_fps_without_camera_motion_videos_transforms.yml')

for database in yml['databases']:
    paths = []
    for files in database['videos']['files']:
        paths.append(os.path.join(
            servers[server]['database']['location'], 
            database['prefix'], 
            database['videos']['prefix'], 
            files)
        )

    # for idx in range(len(paths)):
    composes = []
    for dict_list in database['transforms']:
        composes.append(dict_list_to_compose(dict_list)) # generate compose transform from dict
    consecutive_sequences = ConsecutiveSequences(
        paths=paths, 
        max_seq=10, 
        seq_stride=1, 
        seq_len=1, 
        transforms=composes, 
        verbose=True
    ) # generate iterator
        
    for cs, vid_idx, frame_idx in consecutive_sequences:
        print('\rdatabase: {}, vid_idx: {}, frame_idx: {}'.format(database['name'], vid_idx, frame_idx), end='')
        cv2.imshow('img', cs[0])
        cv2.waitKey()


database: SurgVisDom_test_no_indicators_without_camera_motion, vid_idx: 0, frame_idx: 9

In [3]:
# homography visualization example
yml = load_yaml(path='configs/high_fps_without_camera_motion_videos_homography_transforms.yml')

for database in yml['databases']:
    paths = []
    for files in database['videos']['files']:
        paths.append(os.path.join(
            servers[server]['database']['location'],
            database['prefix'], 
            database['videos']['prefix'], 
            files)
        )

    composes = []
    for dict_list in database['transforms']:
        composes.append(dict_list_to_compose(dict_list)) # generate compose transform from dict
    consecutive_sequences = ConsecutiveSequences(
        paths=paths, 
        seq_stride=1, 
        seq_len=1, 
        transforms=composes,
        verbose=True
    ) # generate iterator
    
    for cs, vid_idx, frame_idx in consecutive_sequences:
        print('\rdatabase: {}, vid_idx: {}, frame_idx: {}'.format(database['name'], vid_idx, frame_idx), end='')
        composes[vid_idx].transforms[-1].visualize(cs[0]) # RandomEdgeHomography transform has visualization tools (last transform in .yml)

        cv2.imshow('wrp', cs[0]['wrp'])
        cv2.imshow('img_crp', cs[0]['img_crp'])
        cv2.imshow('wrp_crp', cs[0]['wrp_crp'])
        cv2.waitKey()

database: SurgVisDom_test_no_indicators_without_camera_motion, vid_idx: 0, frame_idx: 0

NameError: name 'compose' is not defined